Importing my functions.py file

In [87]:
from functions import *

First, I will load all the 6000 tsv files into one final dataset.

In [88]:
directory = "/Users/ricca/Desktop/ADM/files_tsv"
output_file = "/Users/ricca/Desktop/ADM-HW3/grande.tsv"
dataset = create_store_tsv(directory, output_file)

In [89]:
dataset.head(5)

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,"3D visualisation and animation play a role in many areas, and the popularity of these media just keeps growing. Digital animation provides the eye-catching special effects in the 21st century's favourite films and television shows; 3D design is also essential to everyday work in everything from computer games development, online virtual world development and industrial design to marketing, product design and architecture.GCU's programme in 3D Design for Virtual Environments will help you develop the skills to thrive in a successful career as a visual designer. The programme is practical and career-focused, oriented towards current industry needs, technology and practice. No prior knowledge of 3D design is required.",September,Please see the university website for further information on fees for this course.,MSc,1 year full-time,Glasgow,United Kingdom,On Campus,www.findamasters.com/masters-degrees/course/3d-design-for-virtual-environments-msc/?i93d2645c19223
1,"Agricultural, Environmental and Resource Economics - MSc (Agriculture and Forestry)",University of Helsinki,International Masters Degree Programmes,Full time,"Goal of the pro­grammeAre you looking forward to a future as an expert in Agricultural, Environmental and Resource Economics? As a graduate of our Master’s Programme in Agricultural, Environmental and Resource Economics, you can find employment in the national or international market, for example at universities, research institutes, the public sector or in business, or you can become a self-employed entrepreneur.The Viikki Campus offers optimal resources for studying the unique range of subjects offered by our programme. Upon graduating you will be a professional in applied economics in agricultural, environmental and resource-focused fields. You will be well versed in topics such as climate policy, sustainable agriculture and food security.",September,Tuition fee per year (non-EU/EEA students): 15000 €,MSc,2 years,Helsinki,Finland,On Campus,www.findamasters.com/masters-degrees/course/agricultural-environmental-and-resource-economics-msc-agriculture-and-forestry/?i312d6223c45168
2,Energy Policy and Finance (MSc),University of St Andrews,Interdisciplinary Studies,Full time&Part time,"The MSc in Energy Ethics explores how we balance our energy demands with our concerns about anthropogenic climate change. How can we make sense of the complexity of our energy systems and address challenges of energy security and equity? What can we do to create a better energy future for us all?The challenge of creating a better energy future for us all is both urgent and complex. The MSc in Energy Ethics has been specifically designed to match and address this complexity. It brings together academic experts from across arts, humanities, social and natural sciences to offer an in-depth, interdisciplinary understanding of global, national and local energy challenges.",September,"Home£13,470Overseas£27,230",MSc,"1 year full time, 2 year part time",St Andrews,United Kingdom,On Campus,www.findamasters.com/masters-degrees/course/energy-policy-and-finance-msc/?i350d7295c70061
3,Aircraft Design option - Aerospace Vehicle Design (MSc),Cranfield University,Aerospace,Full time,"Study at Cranfield UniversityTo design modern efficient aircraft requires a complex combination of aerodynamic performance, lightweight durable structures and advanced systems engineering. This specialistMSc Aerospace Vehicle Designoption explores how different structural and systems elements can be designed and integrated using up-to-date methods and techniques.Who is it for?This option is suitable for those students wishing to gain an overview of the whole aircraft design process as well as the design of aircraft str

Before going on, to protect the original dataset, let's create a copy of it, deleting the 21 links corrupted that gives us empty values for the variables.

In [90]:
df = dataset.copy()
print(df.shape)
df = df.dropna(subset=['courseName'], axis=0)
print(df.shape)

(6000, 13)
(5979, 13)


# 2. Search Engine
### 2.0.0 Preprocessing the text
First, you must pre-process all the information collected for each MSc by:

  * Removing stopwords
  * Removing punctuation
  * Stemming
  * Anything else you think it's needed
---

Now, as the dataset is loaded, I will apply some preprocessing to all columns. This pre-processing includes, tokenizing, standardazing the text (lower case and removing stop words), and stemming the text of all columns.

In [91]:
desired_columns = [x for x in df.columns if x != "fees"]
for column in desired_columns:
    df[column+"_clean"] = df[column].apply(lambda x: preprocessing(x))

In [92]:
df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,...,facultyName_clean,isItFullTime_clean,description_clean,startDate_clean,modality_clean,duration_clean,city_clean,country_clean,administration_clean,url_clean
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,"3D visualisation and animation play a role in many areas, and the popularity of these media just keeps growing. Digital animation provides the eye-catching special effects in the 21st century's favourite films and television shows; 3D design is also essential to everyday work in everything from computer games development, online virtual world development and industrial design to marketing, product design and architecture.GCU's programme in 3D Design for Virtual Environments will help you develop the skills to thrive in a successful career as a visual designer. The programme is practical and career-focused, oriented towards current industry needs, technology and practice. No prior knowledge of 3D design is required.",September,Please see the university website for further information on fees for this course.,MSc,1 year full-time,Glasgow,...,"[school, engin, built, environ]","[full, time]","[visualis, anim, play, role, mani, area, popular, media, keep, grow, digit, anim, provid, eye, catch, special, effect, favourit, film, televis, show, design, also, essenti, everyday, work, everyth, comput, game, develop, onlin, virtual, world, develop, industri, design, market, product, design, architectur, programm, design, virtual, environ, help, develop, skill, thrive, success, career, visual, design, programm, practic, career, focus, orient, toward, current, industri, need, technolog, practic, prior, knowledg, design, requir]",[septemb],[msc],"[year, full, time]",[glasgow],"[unit, kingdom]",[campu],"[www, findamast, com, master, degre, cours, design, virtual, environ, msc]"
1,"Agricultural, Environmental and Resource Economics - MSc (Agriculture and Forestry)",University of Helsinki,International Masters Degree Programmes,Full time,"Goal of the pro­grammeAre you looking forward to a future as an expert in Agricultural, Environmental and Resource Economics? As a graduate of our Master’s Programme in Agricultural, Environmental and Resource Economics, you can find employment in the national or international market, for example at universities, research institutes, the public sector or in business, or you can become a self-employed entrepreneur.The Viikki Campus offers optimal resources for studying the unique range of subjects offered by our programme. Upon graduating you will be a professional in applied economics in agricultural, environmental and resource-focused fields. You will be well versed in topics such as climate policy, sustainable agriculture and food security.",September,Tuition fee per year (non-EU/EEA students): 15000 €,MSc,2 years,Helsinki,...,"[intern, master, degre, programm]","[full, time]","[goal, pro, grammear, look, forward, futur, expert, agricultur, environment, resourc, econom, graduat, master, programm, agricultur, environment, resourc, econom, find, employ, nation, intern, market, exampl, univers, research, institut, public, sector, busi, becom, self, employ, entrepreneur, viikki, campu, offer, optim, resourc, studi, uniqu, rang, subject, offer, programm, upon, graduat, profession, appli, econom, agricultur, environment, resourc, focus, field, well, vers, topic, climat, polici, sustain, agricultur, food, secur]",[septemb],[msc],[year],[helsinki],[finland],[campu],"[www, findamast, com, master, degre, cours, agricultur, environment, resourc, econom, msc, agricultur, forestri]"
2,Energy Policy and Finance (MSc),University of St Andrews,Interdisciplinary Studies,Full time&Part time,"The MSc in Energy Ethics explores how we balance our energy demands with our concerns about anthropogenic climate change. How can we make sense of the co

### 2.0.1 Preprocessing the fees column
Moreover, we want the field `fees` to collect numeric information. As you will see, you scraped textual information for this attribute in the dataset: sketch whatever method you need (using regex, for example, to find currency symbol) to collect information and, in case of multiple information, retrieve only the highest fees. Finally, once you have collected numerical information, you likely will have different currencies: this can be chaotic, so let chatGPT guide you in the choice and deployment of an API to convert this column to a common currency of your choice (it can be USD, EUR or whatever you want). Ultimately, you will have a `float` column renamed `fees (CHOSEN COMMON CURRENCY)`.

-----

Now, it is time to do some preprocessing with the fees column. We have to ensure that all the fees are in Euro (decided) and the column is of dtype float. So, let's do it!

First, I will replace all the entries that are prompting to contact the university or visit webpage. Then, I will see what currency symbols have been used through out the dataset so that I can desing my regex accordingly.

In [33]:
if_website_or_contact_replace(df, "fees")
df["fees_clean"] = df["fees_clean"].apply(lambda x: "Not Available" if pd.isnull(x) else x)
symbols_list = extract_symbols(df)
symbols_list

['$', '€', '£']

Now, from all of the text in each row of the `fees` column, I will only keep the max amount and the currency name.

In [34]:
df["fees_clean"] = df["fees_clean"].apply(lambda x: extract_numeric_and_currency(x))
df.loc[:, ["fees","fees_clean"]]

,fees,fees_clean
0,Please see the university website for further information on fees for this course.,Not Available
1,Tuition fee per year (non-EU/EEA students): 15000 €,15000.0 EUR
2,"Home£13,470Overseas£27,230",27230.0 GBP
3,Please see the university website for further information on fees for this course.,Not Available
4,Please see the university website for further information on fees for this course.,Not Available
...,...,...
5995,Please see the university website for further information on fees for this course.,Not Available
5996,UK Fees: 2022/23 fees TBC*;2021/22 fees - 10400International Fees: 2022/23 fees TBC*;2021/22 fees - 17900,17900.0 EUR
5997,Please see the university website for further information on fees for this course.,Not Available
5998,Please see the university website for further information on fees for this course.,Not Available


Now, I will create two more columns and use these columns to make make a new column FEE_EUR where all the fees are in EUR.

In [35]:
# Apply extraction functions to create new columns
df['numeric_value'] = df['fees_clean'].apply(extract_numeric_value)
df['currency_name'] = df['fees_clean'].apply(extract_currency_name)

In [ ]:
tqdm.pandas()
def apply_convert_to_eur(row):
    return convert_to_eur(row['numeric_value'], row['currency_name'])
df['FEE_EUR'] = df.progress_apply(apply_convert_to_eur, axis=1)

Finally, I will delete the extra columns. Namely: `fees_clean`, `numeric_value`, `currency_name`

In [ ]:
df.columns
columns_to_drop = ["fees_clean", "numeric_value", "currency_name"]
df = df.drop(columns=columns_to_drop)

### 2.1. Conjunctive query

#### 2.1.1 Create your index!
Before building the index,

- Create a file named `vocabulary`, in the format you prefer, that maps each word to an integer (`term_id`).

In [93]:
vocabulary = set() # set to remove duplicates and efficiency
# I will only used the description column as stated in the HW
df.description_clean.dropna().apply(lambda row: [vocabulary.add(word) for word in row])
vocabulary = list(vocabulary)

The variable vocabulary contains, in a list, all the words used in the description column. The question requires us to make a file named vocabulary that *maps each word to an integer (term_id).*

In [94]:
# maps each word to an integer (term_id)
vocab_with_index = dict()
unique_id = 0
for word in vocabulary:
  vocab_with_index[word] = unique_id
  unique_id+=1
vocab_with_index

with open("./vocabulary.json", "w") as file: # storing for reusability
    json.dump(vocab_with_index, file)

In [95]:
vocab_with_index = json.load(open("./vocabulary.json", "r")) # loading for use

Then, the first brick of your homework is to create the Inverted Index. It will be a dictionary in this format:

{
term_id_1:[document_1, document_2, document_4],
term_id_2:[document_1, document_3, document_5, document_6],
...}

To create the inverted index, I can use the procedure used in the lab as follows:

In [96]:
Terms = pd.DataFrame(data=vocab_with_index.keys(), columns=['Term']) # from lab
tqdm.pandas() # to create partial inverted index, as shown in lab
Terms['reverse'] = Terms.Term.progress_apply(lambda item: list(df[df.description_clean.apply(lambda row: isinstance(row, list) and item in row)].index))
Terms.head()
# inverted_index = dict(zip(Terms.index, Terms['reverse']))

  0%|          | 14/9704 [00:00<01:10, 137.75it/s]

100%|██████████| 9704/9704 [00:53<00:00, 182.20it/s]


,Term,reverse
0,analys,"[76, 77, 78, 116, 125, 144, 150, 167, 215, 235, 252, 268, 279, 301, 352, 401, 430, 440, 453, 479, 485, 488, 489, 518, 532, 554, 602, 620, 645, 670, 672, 698, 703, 727, 745, 750, 762, 765, 769, 791, 792, 796, 812, 834, 840, 886, 944, 964, 1029, 1041, 1048, 1051, 1054, 1219, 1224, 1228, 1235, 1248, 1249, 1263, 1327, 1328, 1344, 1375, 1415, 1470, 1476, 1524, 1548, 1597, 1599, 1610, 1611, 1617, 1624, 1635, 1659, 1671, 1680, 1692, 1726, 1730, 1776, 1782, 1792, 1799, 1800, 1802, 1806, 1807, 1849, 1850, 1852, 1857, 1866, 1872, 1874, 1916, 1918, 1936, ...]"
1,disintermedi,[257]
2,migrat,"[2225, 2660, 2661, 2662, 2663, 2664, 2896, 3263, 3375, 3404, 3462, 3463, 3464, 3480, 3751, 4030, 4157, 4332, 4335, 4344, 4380, 4388, 4633, 5095, 5121]"
3,focusedphysiolog,[232]
4,ibhc,[698]


Or, I can use the function that I create to make an inverted list. Here's how:

In [97]:
inverted_index = inverted_index(vocab_with_index, df)

I will now store this inverted index locally

In [98]:
with open("./inverted_index_1.json", "w") as file: # storing for reusability
    json.dump(inverted_index, file)
inverted_index = json.load(open("./inverted_index_1.json", "r")) # loading for use

### 2.1.2 Execute the query
Given a query input by the user, for example:

`advanced knowledge`

The Search Engine is supposed to return a list of documents.

---

For this task, I will first take the input from the user and then normalize the query using the same technique I used to preprocess the description column.

In [99]:
# user_input = input("Add your Input")
user_input = "data science"
normalized_query = normalize_query(user_input)

Now, I will extract the term_id from the vocabulary with the index in order to utilize it for the inverted index

In [100]:
term_ids = get_term_id_from_query(vocab_with_index, normalized_query)

In [101]:
new_dataset = search_engine(term_ids, df, inverted_index)
new_dataset

,courseName,universityName,description
0,Digital Forensics and Cyber Investigation (with Advanced Practice) MSc,Teesside University,"With a clear focus on forensics and industry practices, this course equips you with the knowledge and skills to become a professional digital or cyber analyst, or investigator.Digital forensics is a key part of your studies, alongside aspects of computer science. You apply core computer science principles within the context of a cyber investigation, considering how they can be used to collect, examine and interpret digital evidence. There is a strong emphasis on interpreting raw digital data and developing reverse engineering skills. In practical sessions, you use industry-standard software and practices, understanding the intelligence aspects of evidence and intelligence acquired from examining material."
1,Digital Forensics and Cyber Investigation MSc,Teesside University,"With a clear focus on forensics and industry practices, this course equips you with the knowledge and skills to become a professional digital or cyber analyst, or investigator.Digital forensics is a key part of your studies, alongside aspects of computer science. You apply core computer science principles within the context of a cyber investigation, considering how they can be used to collect, examine and interpret digital evidence. There is a strong emphasis on interpreting raw digital data and developing reverse engineering skills. In practical sessions, you use industry-standard software and practices, understanding the intelligence aspects of evidence and intelligence acquired from examining material."
2,Applied Analytical Chemistry MSc,University College London,"Register your interest in graduate study at UCLAnalytical chemistry underpins many important commercial enterprises from jet engine development to food production and is applied to other academic disciplines such as Earth sciences, medicine, archaeology, pharmacy and forensics. This MSc seeks to train the next generation of analytical scientists in state-of-the-art methods and skills to tackle the challenges of this broad range of applications.About this degreeThis programme is designed to provide comprehensive training in analytical chemistry and its implementation. A thorough understanding of error analysis, data processing and data presentation will be at the foundation of this programme."
3,Applied Artificial Intelligence and Data Analytics MSc,University of Bradford,"*SCHOLARSHIPS AVAILABLE*This new MSc programme has been designed in response to the shortage of Artificial Intelligence (AI) and Data Science (DS) specialists in the UK. It will give you the skills and professional insight you need to launch a career in these diverse and fast-growing sectors.A number of£10,000 scholarshipswill be offered to Home students attending this programme, as part of a £1.3m investment by Office for Students (OfS), Department for Culture, Media and Sport (DCMS), Office for Artificial Intelligence and industry contributions."
4,Environmental Data Science and Analytics - MSc,University of Leeds,"As global discussions are increasingly focused on the environment and sustainability, the need to understand and interpret both built and natural environmental data is more crucial than ever. Governments, organisations, and industries across the world are seeking individuals who can leverage data to make informed, sustainable decisions.Our Environmental Data Science and Analytics MSc is designed to give you a fuller understanding of data and what appropriate interpretation and insight generation means in various environmental contexts. Throughout, you'll develop a broad understanding of environmental issues and how data driven analysis can help address these."
...,...,...,...
357,Design Informatics MSc,University of Edinburgh,"Programme DescriptionDesign informatics combines data science with design thinking in a context of critical enquiry to create compelling and meaningful new dat

## 2.2 Conjunctive query & Ranking score

For the second search engine, given a query, we want to get the top-k (the choice of k it's up to you!) documents related to the query. In particular:

Find all the documents that contain all the words in the query. Sort them by their similarity with the query.

Return in output k documents, or all the documents with non-zero similarity with the query when the results are less than k. You must use a heap data structure (you can use Python libraries) for maintaining the top-k documents.

To solve this task, you must use the tfIdf score and the Cosine similarity. The field to consider is still the description.

##### 2.2.1 Make an Inverted index

Below, I will make an inverted index which contains as keys the unique term_id of the word, and as values, its corresponding occurences and tf-idf values in the dataset's row.

In [102]:
# will help with the search engine and inverted index. It contains as keys the words and as values a list which 
# represents the occurences of that word in our dataset
word_and_appearances = dict(zip(Terms["Term"], Terms['reverse']))

# Our inverted_index with tf-idf value as well.
inverted_index_tf_idf = inverted_index_tfidf(word_and_appearances, df)

In [103]:
with open("./inverted_index_tfidf.json", "w") as file: # storing for reusability
    json.dump(inverted_index, file)
inverted_index = json.load(open("./inverted_index_tfidf.json", "r")) # loading for use

#### 2.2.2 Execute the query
In this new setting, given a query, you get the proper documents (i.e., those containing all the query's words) and sort them according to their similarity to the query. For this purpose, as the scoring function, we will use the Cosine Similarity concerning the tfIdf representations of the documents.

---

First, I will take the user input and normalize it.

In [104]:
# user_input = "input("Add your Input")"
user_input = "data science"
normalized_query = normalize_query(user_input)
normalized_query

['data', 'scienc']

I will now compute the intersection list of query and documents.

In [105]:
term_ids = get_term_id_from_query(vocab_with_index, normalized_query)
appearances = list()
[appearances.append(inverted_index[str(term)]) for term in term_ids]
# initialising the set with the value of the first list of appearances in the list of appearances of the word
intersection_list = set(appearances[0])
for appearance in appearances[1:]: # for the rest of the terms in the list of term ids
    intersection_list.intersection_update(appearance)
intersection_list = sorted(list(intersection_list))
intersection_list

[62,
 67,
 68,
 70,
 77,
 92,
 99,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 114,
 115,
 116,
 117,
 118,
 124,
 165,
 171,
 209,
 213,
 240,
 243,
 249,
 250,
 252,
 253,
 254,
 267,
 324,
 334,
 337,
 353,
 355,
 423,
 486,
 487,
 495,
 518,
 524,
 526,
 527,
 531,
 532,
 533,
 535,
 536,
 537,
 631,
 646,
 670,
 721,
 736,
 739,
 757,
 771,
 772,
 773,
 789,
 807,
 838,
 1000,
 1012,
 1187,
 1201,
 1212,
 1242,
 1302,
 1303,
 1305,
 1306,
 1326,
 1328,
 1361,
 1362,
 1398,
 1399,
 1402,
 1404,
 1408,
 1412,
 1419,
 1421,
 1425,
 1451,
 1455,
 1458,
 1462,
 1476,
 1556,
 1720,
 1729,
 1756,
 1763,
 1764,
 1765,
 1766,
 1768,
 1769,
 1771,
 1773,
 1774,
 1775,
 1776,
 1783,
 1784,
 1785,
 1788,
 1789,
 1791,
 1792,
 1793,
 1794,
 1795,
 1796,
 1797,
 1798,
 1799,
 1800,
 1802,
 1803,
 1804,
 1805,
 1806,
 1807,
 1808,
 1809,
 1810,
 1813,
 1814,
 1815,
 1816,
 1817,
 1818,
 1819,
 1820,
 1821,
 1822,
 1824,
 1825,
 1826,
 1828,
 1829,
 1830,
 1831,
 1832,
 1833,
 1835,
 1836,
 1837,


Lets now get the tf idf of the query

In [106]:
query_tfidf = calculate_query_tfidf(normalized_query, word_and_appearances, df)
query_tfidf

{'data': 1.04, 'scienc': 0.74}

In [107]:
similarity_scores = {}
for document_i in intersection_list:
    document_vector = df.at[document_i, "description_clean"] # getting the document vector at the row of interest
    # calculated the tf_idf of the document vector. It will result in words as keys and values as tf-idf scores
    document_tfidf = calculate_document_tfidf(document_vector, word_and_appearances, df) 
    # Compute cosine similarity
    dot_product = 0
    for word in normalized_query:
        if word in document_tfidf:
# computing the dot product of only the query's tfidf score and the tfidf score of that word in the document
            dot_product += query_tfidf[word] * document_tfidf[word] 
    
    norm_doc_i = np.linalg.norm(list(document_tfidf.values())) # computing norm of the doc vector
    norm_query = np.linalg.norm(list(query_tfidf.values()) )# computing norm of the query vector
    
    if norm_doc_i != 0 and norm_query != 0: # only the non zero results
        cosine_similarity_doc_i_query = dot_product / (norm_doc_i * norm_query)
        similarity_scores[document_i] = cosine_similarity_doc_i_query

Using heap data structure to retrieve top 10 similarity scores:

In [108]:
top_10_documents = heapq.nlargest(10, similarity_scores, key=similarity_scores.get)

Finally, bringing the result of the query search.

In [111]:
results = return_results(top_10_documents, similarity_scores, df)
pd.set_option("display.max_colwidth", None)
results

,courseName,universityName,description,Cosine_Similarity
0,"Data Science and its Applications, MSc",University of Greenwich,"Our MSc degree in Data Science and its Applications is designed to provide you with a solid grounding in data science theory and practice.Enter the exciting world of data science whatever your academic background! Our MSc in Data Science and its Applications course has been designed to increase the skilled workforce and diversity in qualified data science experts in the UK market. The programme is tailored to transform professionals from a wide range of backgrounds into accomplished data scientists who are well-placed to enhance their existing careers with an expansive set of data science skills, to move fully into data science roles, or to pursue further data science specialisations.",0.706323
1,Data Science - Master of Science (MS),University of Colorado Boulder,"The on-campus Master of Science in Data Science program focuses on developing knowledge and skills in interdisciplinary and collaborative data science competencies including statistical analysis, data structures and algorithms, data mining, machine learning, big data architecture and data visualization.Data science is a multidisciplinary field that focuses on the extraction of knowledge and insight from large datasets. Data scientists are tasked with using a range of skills in applied mathematics, statistics and computer science, and in domain applications such as information science, geography, business, media and the humanities.",0.627953
2,Data Science MSc,University of Greenwich,"Our MSc in Data Science equips graduates to embark on highly skilled careers in data science, artificial intelligence and machine learning.This specialist Master's in Data Science provides a theoretical knowledge of data science alongside the practical skills that will help you prosper in the jobs market.You'll be exposed to a broad range of topics such as data science, statistics, specialist programming, machine learning and data visualisation.",0.614736
3,Data Science - MSc/PgD/PgC,Cardiff Metropolitan University,"This Master's degree in Data Science is an industry-relevant and popular postgraduate programme of study. Data Science and ""Big Data"" are more important than ever. Graduates trained to garner insights from large data sets, extract patterns and provide actionable information to solve real-world problems are sought after in both the private and public sector.This course will equip you with in-demand theoretical knowledge and practical skills to develop data science systems, use software to analyse and synthesise data, and manage all aspects of data science.Course ContentCompulsory",0.560065
4,Big Data Science and Technology - MSc,University of Bradford,"Develop the skills you need for a career in this fast emerging field of data science.This programme will deepen your understanding of advanced software development, systems for big data analytics, statistical data analysis, data mining, data privacy and security, data visualisation and exploration.It is an interdisciplinary programme, designed for students with a first degree in subjects such as:",0.548488
5,Data Science MSc,Coventry University,"Data is everywhere. As the volume and complexity of data collected continues to grow, there is increasing demand for expertise in data science to support the analysis and visualisation of all this information.The MSc Data Science is a conversion course for graduates from a wide range of disciplines and backgrounds looking to pursue a career, or upskill, in this new and rapidly developing field. Data Scientists are in short supply and there is high demand for data science skills across sectors including business, government, healthcare, science, finance, and marketing.",0.541314
6,Data Science MSc,University of Southampton,"Become a proficient data scientist with a master’s in Data Science at the University of Southampton.You’ll study the latest techniques and te

# 3. Define a new score!

Now it's your turn: build a new metric to rank MSc degrees.

Practically:

- The user will enter a text query. As a starting point, get the query-related documents by exploiting the search engine of Step 2.1.
- Once you have the documents, you need to sort them according to your new score. In this step, you won't have any more to take into account just the description field of the documents; you can use also the remaining variables in your dataset (or new possible variables that you can create from the existing ones or scrape again from the original web-pages). You must use a heap data structure (you can use Python libraries) for maintaining the top-k documents.

N.B.: You have to define a scoring function, not a filter!

The output, must contain:

- courseName
- universityName
- description
- URL
- The new similarity score of the documents with respect to the query

Are the results you obtain better than with the previous scoring function? Explain and compare results.

---

The idea behind the development of the new scoring function is an extension of the concept introduced in *Q2.2* to encompass other variables. This involves assigning varying weights based on the input query. This approach is motivated by the understanding that a user may have diverse interests when searching for information, such as specifying a city, a university name, or the course name. In such cases, if the query aligns with one of these specific variables, it is deemed to be more significant than others. This allows for a more nuanced and personalized scoring system tailored to the user's preferences and priorities.

Initially, we developed a function to efficiently compute weights based on whether the query matches any of the variables. Emphasis is given to the variable that matches the query, assigning higher importance to that specific variable.

After that we use the 'fuzzywuzzy' package, importing Levenshtein Distance to calculate the similarity between the input query and the variable value for each one of them.

In [117]:
import heapq
from fuzzywuzzy import fuzz

def calculate_total_score(query, df, weights):
    total_score = 0
    
    for variable, weight in weights.items():
        if variable in df and isinstance(df[variable], str):
            # Calculate text similarity based on the ratio (using fuzzywuzzy library)
            similarity_score = fuzz.ratio(query, df[variable])
            
            # Normalize the similarity score between 0 and 1
            normalized_similarity = similarity_score / 100.0

            # Calculate the weighted score for the current variable
            variable_score = normalized_similarity * weight

            # Add the variable's score to the total score
            total_score += variable_score

    return total_score

user_query = "data science" 
normalized_query = normalize_query(user_query)
term_ids = get_term_id_from_query(vocab_with_index, normalized_query)
new_dataset = search_engine_full(term_ids, df, inverted_index)

weights = calculate_weights(user_query, new_dataset)

user_query = "data science tampere" 

new_dataset['score'] = new_dataset.apply(lambda row: calculate_total_score(user_query, row, weights), axis=1)
new_dataset.head(1)

heap = []
for index, row in new_dataset.iterrows():
    heapq.heappush(heap, (-row['score'], index))

# Extract results
sorted_indices = [heapq.heappop(heap)[1] for _ in range(len(heap))]

# Create new df sorted
sorted_df = new_dataset.loc[sorted_indices]
sorted_df


,courseName,universityName,facultyName,city,country,description,score
330,Data Science MSc,Lancaster University,Department of Mathematics and Statistics,Lancaster,United Kingdom,"From business and finance to health and medicine, from infrastructure to societal studies, data science plays a vital role in all aspects of the modern world. Our MSc programme will ensure you have an advanced level of skills, knowledge, and experience in this rapidly expanding, highly in-demand field to achieve your career aspirations.",0.417273
254,Data Science,Aalto University,EIT Digital Master School - Aalto University,Tampere,Finland,"Big Data can provide important insights into business processes and uncover opportunities to cut costs and increase efficiency. Furthermore, data analytics enables organisations to effectively target specific audiences and to adapt product characteristics based on customer preferences. It can also be used to improve the decision-making process.Yet, despite the immense potential of Big Data, many companies are blind to the value of the data they create. It is time for that to change - and our Data Science programme equips you with the skills to drive this change.",0.412727
345,Data Science MSc,Nottingham Trent University,School of Science & Technology,Nottingham,United Kingdom,"This course covers the fundamentals of Data Science including Big Data infrastructure, AI and statistical analysis and the mining of data. Development of your understanding of deriving value from the data analytics functions is also a focus of this course.You will learn how to support organisations in improving their data capture, management and analysis processes with a focus on efficiency and profit, whilst adhering to the current highest ethical standards.",0.411818
329,Data Science MSc,University of Chester,Computer Science,Chester,United Kingdom,"A conversion Master’s degree in Data Science, to enable students to make the leap to the fast-growing area of data science.Course overviewThe Data Science Master’s degree is aimed at people who have a technical, mathematical, or engineering background who want to develop their skills into the field of data science. Students who have worked in one of these areas but do not process a degree in the area are welcome to apply.Why study this course with us?The Data Science course is within the Department of Computer Science, who are a forward-thinking innovative department.We work with employers to tailor the course to real-world needs, giving students an in-depth knowledge in the area.",0.408182
328,Data Science MSc,University of Exeter,Computer Science,Exeter,United Kingdom,"OverviewBy studying Data Science at Exeter you will be joining a rapidly growing department that is already among the top ten for computer science in the UK.This programme offers you the flexibility to pursue data science according to your own passions.Designed for those interested in learning the underpinning theory of Data Science together with methods for implementation and application.A large component of the degree involves a research project which is a one-to-one engagement with a mentor from the university who will be an active data scientist and leader in their field.This course is for anyone with a foundational level of coding and mathematical knowledge who wants a thorough grounding in the fundamentals of modern data science as well as its application to real-world problems.Teaching and researchTeaching is mainly delivered by lectures, workshops and online materials. Each module references core and supplementary texts, or material recommended by module deliverers, which provide in depth coverage of the subject and go beyond the lectures.",0.406364
...,...,...,...,...,...,...,...
74,Bioinformatics and Computational Biology MSc,University College Cork,"College of Science, Engineering and Food Science",Cork,Ireland,"Our MSc in Bioinformatics & Computational Biology is a one-year full-time (or two years part-time) taught m